In [2]:
from twarc import Twarc2

In [5]:
import yaml

In [6]:
with open('../bearer_token.yaml') as f:
    bearer_token = yaml.safe_load(f)['bearer_token']

In [8]:
import pandas as pd

In [14]:
tweets = pd.read_csv('../test_data/twacapic-parliamentarians-tryout.csv', engine='python', dtype=str)

In [16]:
ids = tweets['id']

In [17]:
ids

0         1408766774598942720
1         1408764816894873601
2         1379454336170926084
3         1378047272634703874
4         1411421916095827976
                 ...         
357094    1387160260893462528
357095    1403991909190733825
357096    1380088206931202048
357097    1384751733637849088
357098    1380423504110219264
Name: id, Length: 357099, dtype: object

In [50]:
test_ids = ids.sample(1000)
test_ids

133071    1396496915924140037
165979    1393558562006355970
9100      1388812025065836544
138660    1357696050853801988
333976    1384209498001219585
                 ...         
127548    1415326041862770692
347294     879756341191028739
204300    1123543883223379968
92225     1364921905413816320
202846    1382337118823923712
Name: id, Length: 1000, dtype: object

In [51]:
t = Twarc2(bearer_token=bearer_token, connection_errors=1)

In [52]:
lookup = t.tweet_lookup(test_ids.values)

In [53]:
result = [page for page in lookup]

In [54]:
len(result)

10

In [56]:
errors = list()

for page in result:
    page_length = len(page['data'])
    if page_length != 100:
        assert 'errors' in page.keys()
        n = 0
        for error in page['errors']:
            if error['parameter'] == 'ids' and error['resource_type'] == 'tweet':
                assert error['value'] in test_ids.values
                errors.append(error)
                n += 1
        assert page_length == 100 - n
            

In [57]:
import ndjson

In [59]:
with open('deleted_tweets.ndjson', 'w') as f:
    ndjson.dump(errors, f)